# Frame Check - Jay
Copyright © 2020 AA

### **不要選到別人的資料夾, 避免同時標注同一份檔案**

In [8]:
import os, shutil, joblib, datetime, time, glob
from pathlib import Path
from pigeon import annotate
from ipyfilechooser import FileChooser
from IPython.display import display, Image
import pandas as pd
def checkandmkdir(fpath):
    dirname = os.path.dirname(fpath)
    os.makedirs(dirname, exist_ok=True)


## 當前Label狀態

In [9]:
haslabellist=[]
nolabellist=[]

for f in glob.glob('/mnt/hdd/TrData/autolabel*/*_bbox'):
    if not os.path.isdir(f):
        continue
        
    flag = os.path.join(f,'has_review.flag')
    if os.path.exists(flag):
        haslabellist.append(f)
    else:
        nolabellist.append(f)
#print('== Has label ==:')
#for f in sorted(haslabellist):
#    print('\t',f)
    
print('== No label ==:')
for f in sorted(nolabellist):
    print('\t',f)    

== No label ==:
	 /mnt/hdd/TrData/autolabel_1102_ppen/N01-3F 05L F68-05 78.30 R11.12_bbox
	 /mnt/hdd/TrData/autolabel_1106_ppen_1/N01-3F 05L F68-05 78.30 R11.12_bbox
	 /mnt/hdd/TrData/autolabel_1106_ppen_2/N01-3F 05L F68-05 78.30 R11.12_bbox


## 選擇資料夾

In [10]:
root='/mnt/hdd/TrData'
fc = FileChooser(root)
fc.show_hidden = True
fc.use_dir_icons = True
display(fc)

FileChooser(path='/mnt/hdd/TrData', filename='', show_hidden='True')

In [11]:
assert fc.selected_path, 'Please Select a folder'
assert 'bbox' in fc.selected_path, '選錯資料夾, 請選資料夾名稱有bbox的'
folderpath = fc.selected_path
IMG_DATE = '/'.join(folderpath.split('/')[-2:]) #os.path.basename(folderpath)#'0430'

time.sleep(1)
#確認Label ＯＫ
conf = input(f'是否標註{IMG_DATE}的資料 Y/N')
if conf=='Y':
    pass
else:
    raise BaseException("取消標註")


flagpath = os.path.join(folderpath, f'has_review.flag')
assert os.path.exists(root), f'{root} Not Exist'
assert not os.path.exists(flagpath), 'Flag存在, 這份資料標注過了'



是否標註autolabel_1102_ppen/N01-3F 05L F68-05 78.30 R11.12_bbox的資料 Y/NY


## 標註物件類別

In [12]:
#注意! 重新執行會清空已標註檔案

#image path
img_dict = {}
label_dict ={}
klist = os.listdir(folderpath)
klist = list(filter(lambda k: os.path.isdir(os.path.join(folderpath,k)) ,klist))
for k in klist:
    #os.path.isdir(os.path.join(folderpath,k[0]))
    img_dict[k]= glob.glob(os.path.join(folderpath, f'{k}/*.jpg'))
for k in klist:
    print('====',f'{k}','====')
    #labeling
    imgs = img_dict[k]
    annotations = annotate(
      imgs,
      options=['OK','NG'],
      display_fn=lambda filename: display(Image(filename))
    )
    label_dict[k]=annotations

==== HAND ====


HTML(value='0 examples annotated, 126 examples left')

Output()

## 標注完畢, 刪除NG影像
- 讀取train_raw.txt
- 刪去不要的數據
- 儲存train.txt

In [ ]:
print(f'確認資料夾: {IMG_DATE}')
print(f'確認影像: {folderpath}')
imgqty = sum(list(map(lambda x: len(x), list(img_dict.values()))))
labelqty = sum(list(map(lambda x: len(x), list(label_dict.values()))))
print(f'影像數量: {imgqty}')
print(f'標註數量: {labelqty}')

nglist = []
for k in list(label_dict.values()):
    for path, ret in k.values():
        if ret =='NG':
            newpath = path.split('/')
            newpath[5] = newpath[5][:-5]
            newpath_str = '/'.join(newpath)        
            nglist.append(newpath_str)
        pass
    pass
    
print(f'刪除數量: {len(nglist)}')

time.sleep(1)
#確認Label ＯＫ
conf = input('確認？Y/N')
print(conf)
if conf=='Y':
    print('Save Label')
    joblib.dump(label_dict, os.path.join(folderpath, f'label_dict.pkl'))
    flag = open(flagpath, 'a')
    dt= datetime.datetime.today()  
    dt_str = f'{dt:%Y%m%d%H}'     
    flag.write(dt_str) 
    flag.write('\n')
    flag.close()
else:
    print('[Warning] !!! Cancel... !!!')
    
    
#寫入train.txt
##FIXME!!! -> 每次都複寫train.txt, 所以只會保留一個資料夾的過濾資料, 其他都會被覆寫
if conf=='Y':
    print('Confirm and save')
    trpath = Path(folderpath)
    
    if os.path.exists(os.path.join(trpath.parent,'train.txt')):
        train_df = pd.read_csv(os.path.join(trpath.parent,'train.txt'), header=None)
    else:
        train_df = pd.read_csv(os.path.join(trpath.parent,'train_raw.txt'), header=None)
        
    trainNew_df = train_df[~train_df[0].isin(nglist)]
    trainNew_df.to_csv(os.path.join(trpath.parent,'train.txt'), index=False, header=None)
    print(f'刪除數量: {len(nglist)}')
    print('原始數量:', train_df.shape, 'new qty:',trainNew_df.shape)
else:
    print('Not confirm yet')
        